In [ ]:
from IPython.display import clear_output
!pip install imutils
clear_output()
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
path = "/BraTS20-Invariant-feature/BraTS2020_ResNet50_Split/"
os.chdir(path)
os.getcwd()
IMG_PATH = "/BraTS20-Invariant-feature/BraTS2020_ResNet50/"

In [ ]:
import numpy as np
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [ ]:
!apt-get install tree
!mkdir TRAIN VAL TEST TRAIN/HGG TRAIN/LGG VAL/HGG VAL/LGG TEST/HGG TEST/LGG
!tree -d

In [ ]:
# split the data by train/val/test
for CLASS in os.listdir(IMG_PATH):
    if not CLASS.startswith('.'):
        IMG_NUM = len(os.listdir(IMG_PATH + CLASS))
        for (n, FILE_NAME) in enumerate(os.listdir(IMG_PATH + CLASS)):
            img = IMG_PATH + CLASS + '/' + FILE_NAME
            if not FILE_NAME.startswith('.'):
              if n < 5:
                  shutil.copy(img, path + 'TEST/' + CLASS.upper() + '/' + FILE_NAME)
              elif n < 0.8*IMG_NUM:
                  shutil.copy(img, path + 'TRAIN/'+ CLASS.upper() + '/' + FILE_NAME)
              else:
                  shutil.copy(img, path + 'VAL/'+ CLASS.upper() + '/' + FILE_NAME)

In [ ]:
def load_data(dir_path, img_size=(100,100)):
    """
    Load resized images as np.arrays to workspace
    """
    X = []
    y = []
    i = 0
    labels = dict()
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            labels[i] = path
            for file in os.listdir(dir_path + path):
                if not file.startswith('.'):
                    img = cv2.imread(dir_path + path + '/' + file)
                    X.append(img)
                    y.append(i)
            i += 1
    X = np.array(X)
    y = np.array(y)
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return X, y, labels



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    cm = np.round(cm,2)
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
TRAIN_DIR = 'TRAIN/'
TEST_DIR = 'TEST/'
VAL_DIR = 'VAL/'
IMG_SIZE = (224,224)

# use predefined function to load the image data into workspace
X_train, y_train, labels = load_data(TRAIN_DIR, IMG_SIZE)
X_test, y_test, _ = load_data(TEST_DIR, IMG_SIZE)
X_val, y_val, _ = load_data(VAL_DIR, IMG_SIZE)

In [ ]:
def plot_samples(X, y, labels_dict, n=50):
    """
    Creates a gridplot for desired number of images (n) from the specified set
    """
    for index in range(len(labels_dict)):
        imgs = X[np.argwhere(y == index)][:n]
        j = 10
        i = int(n/j)

        plt.figure(figsize=(15,6))
        c = 1
        for img in imgs:
            plt.subplot(i,j,c)
            plt.imshow(img[0])

            plt.xticks([])
            plt.yticks([])
            c += 1
        plt.suptitle('Tumor: {}'.format(labels_dict[index]))
        plt.show()

plot_samples(X_train, y_train, labels, 10)

In [ ]:
def crop_imgs(set_name, add_pixels_value=0):
    """
    Finds the extreme points on the image and crops the rectangular out of them
    """
    set_new = []
    for img in set_name:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)

        # threshold the image, then perform a series of erosions +
        # dilations to remove any small regions of noise
        thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)

        # find contours in thresholded image, then grab the largest one
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        # find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = add_pixels_value
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        set_new.append(new_img)

    return np.array(set_new)

In [ ]:
img = cv2.imread('/BraTS20-Invariant-feature/BraTS2020_ResNet50/HGG/BraTS20_Training_001_t2.jpg')
img = cv2.resize(
            img,
            dsize=IMG_SIZE,
            interpolation=cv2.INTER_CUBIC
        )
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# threshold the image, then perform a series of erosions +
# dilations to remove any small regions of noise
thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

# find contours in thresholded image, then grab the largest one
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)

# find the extreme points
extLeft = tuple(c[c[:, :, 0].argmin()][0])
extRight = tuple(c[c[:, :, 0].argmax()][0])
extTop = tuple(c[c[:, :, 1].argmin()][0])
extBot = tuple(c[c[:, :, 1].argmax()][0])

# add contour on the image
img_cnt = cv2.drawContours(img.copy(), [c], -1, (0, 255, 255), 4)

# add extreme points
img_pnt = cv2.circle(img_cnt.copy(), extLeft, 8, (0, 0, 255), -1)
img_pnt = cv2.circle(img_pnt, extRight, 8, (0, 255, 0), -1)
img_pnt = cv2.circle(img_pnt, extTop, 8, (255, 0, 0), -1)
img_pnt = cv2.circle(img_pnt, extBot, 8, (255, 255, 0), -1)

# crop
ADD_PIXELS = 0
new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(141)
plt.imshow(img)
plt.xticks([])
plt.yticks([])
plt.title('Step 1. Get the original image')
plt.subplot(142)
plt.imshow(img_cnt)
plt.xticks([])
plt.yticks([])
plt.title('Step 2. Find the biggest contour')
plt.subplot(143)
plt.imshow(img_pnt)
plt.xticks([])
plt.yticks([])
plt.title('Step 3. Find the extreme points')
plt.subplot(144)
plt.imshow(new_img)
plt.xticks([])
plt.yticks([])
plt.title('Step 4. Crop the image')
plt.show()

In [ ]:
# apply this for each set
X_train_crop = crop_imgs(set_name=X_train)
X_val_crop = crop_imgs(set_name=X_val)
X_test_crop = crop_imgs(set_name=X_test)

In [ ]:
plot_samples(X_train_crop, y_train, labels, 10)

In [ ]:
def save_new_images(x_set, y_set, folder_name):
    i = 0
    for (img, imclass) in zip(x_set, y_set):
        if imclass == 0:
            cv2.imwrite(folder_name+'LGG/'+str(i)+'.jpg', img)
        else:
            cv2.imwrite(folder_name+'HGG/'+str(i)+'.jpg', img)
        i += 1

In [ ]:
path = "/BraTS20-Invariant-feature/BraTS2020_ResNet50_Crop/"
os.chdir(path)
os.getcwd()

In [ ]:
import os
import cv2

def save_new_images(images, labels, folder_name):
    for i, (image, label) in enumerate(zip(images, labels)):
        label_folder = os.path.join(folder_name, 'HGG' if label == 1 else 'LGG')
        os.makedirs(label_folder, exist_ok=True)

        image_filename = os.path.join(label_folder, f'image_{i}.png')
        cv2.imwrite(image_filename, image)

# Create directories and save images
save_new_images(X_train_crop, y_train, folder_name='TRAIN_CROP/')
save_new_images(X_val_crop, y_val, folder_name='VAL_CROP/')
save_new_images(X_test_crop, y_test, folder_name='TEST_CROP/')

In [ ]:
import keras
from keras.layers import BatchNormalization

NUM_CLASSES = 1

# use the pre-trained ResNet50 model
resnet50_x = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

resnet50 = Sequential()
resnet50.add(resnet50_x)
resnet50.add(layers.Dropout(0.3))
resnet50.add(layers.Flatten())
resnet50.add(layers.Dropout(0.5))
resnet50.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))
#resnet50.add(BatchNormalization())

resnet50.layers[0].trainable = False

resnet50.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

resnet50.summary()

In [ ]:
TRAIN_DIR = 'TRAIN_CROP/'
VAL_DIR = 'VAL_CROP/'

# No data augmentation for training set
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# No data augmentation for validation set
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=64,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    color_mode='rgb',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary',
    seed=RANDOM_SEED
)

In [ ]:
import time

start = time.time()

resnet50_history = resnet50.fit(
    train_generator,
    steps_per_epoch=5,
    epochs=150,
    validation_data=validation_generator,
    validation_steps=3,
)

end = time.time()
print(end - start)

In [ ]:
import pandas as pd

# Assuming resnet50_history is a History object returned by the fit method
resnet50_accuracy = resnet50_history.history['accuracy']
resnet50_loss = resnet50_history.history['loss']
resnet50_val_accuracy = resnet50_history.history['val_accuracy']
resnet50_val_loss = resnet50_history.history['val_loss']

# Create a DataFrame
df = pd.DataFrame({
    'Accuracy': resnet50_accuracy,
    'Loss': resnet50_loss,
    'Validation Accuracy': resnet50_val_accuracy,
    'Validation Loss': resnet50_val_loss
})

# Save to Excel file
df.to_excel('resnet50_history.xlsx', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the Excel file
df = pd.read_excel('resnet50_history.xlsx')

# Plot training and validation accuracy
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(df['Accuracy'], label='Training Accuracy')
plt.plot(df['Validation Accuracy'], label='Validation Accuracy')
plt.title('BraTS2020-ResNet50')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(df['Loss'], label='Training Loss')
plt.plot(df['Validation Loss'], label='Validation Loss')
plt.title('BraTS2020-ResNet50')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
y_pred = resnet50.predict(validation_generator)
y_pred_binary = np.round(y_pred)
y_true = validation_generator.classes
conf_mat = confusion_matrix(y_true, y_pred_binary)
print("Confusion Matrix:")
print(conf_mat)

In [ ]:
#sensitivity = cm_ori[0,0]/(cm_ori[0,0]+cm_ori[0,1])
#specificity = cm_ori[1,1]/(cm_ori[1,0]+cm_ori[1,1])

conf_mat_shape = (2, 2)
desired_values = [11, 4, 8, 33]
conf_mat1 = np.reshape(desired_values, conf_mat_shape)

In [ ]:
sensitivity = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
print('Sensitivity : ', sensitivity )

specificity = conf_mat1[1,1]/(conf_mat1[1,0]+conf_mat1[1,1])
print('Specificity : ', specificity)

In [ ]:
import matplotlib.colors as mcolors
import seaborn as sns
class_labels = ['LGG', 'HGG']

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat1, annot=True, cmap='Blues', fmt='d', cbar=False, xticklabels=class_labels, yticklabels=class_labels, annot_kws={"size": 25, "weight": "bold"})
ax = plt.gca()
ax.set_xticklabels(class_labels, fontsize=16, weight='bold')
ax.set_yticklabels(class_labels, fontsize=16, weight='bold')
plt.xlabel("Predicted", fontsize=14, weight='bold')
plt.ylabel("Actual", fontsize=14, weight='bold')
plt.show()